# Libraries

In [4]:
import os
import csv
import json
import ast
import nltk
from nltk import word_tokenize, ngrams
from os.path import exists
from itertools import islice
import math
from pathlib import Path


import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score, recall_score, roc_curve, precision_score, auc
import sklearn
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
import copy
import pickle


# SVM, SVC, Grid Search
from sklearn.model_selection import GridSearchCV
from sklearn import svm
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC


from sklearn.neural_network import MLPRegressor
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
import pandas as pd

from sklearn.naive_bayes import GaussianNB

# logistic regression LR
from sklearn.linear_model import LogisticRegression

import seaborn as sns
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt

# For speading up SVM 
from sklearnex import patch_sklearn 
patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


# Construct ngram class

In [2]:
'''
- In this class, an example of the ngram of a word is: google: [('g', 'o'), ('o', 'o'), ('o', 'g'), ('g', 'l'), ('l', 'e')]
- The ngram frequencies are calculated without taking into consideration the TLD

'''

class ngram_freq_P4:
    def __init__(self, ngram, P4_mfactor):
        self.ngram = ngram
        self.P4_mfactor = P4_mfactor
        self.ngrams_fdist = None
        
    def create_ngram_scores(self, input_file, output_csv=None) -> dict:
        '''
            Arguments:
                input_file: the file to generate the frequencies from (english dictionary, website)
                output_csv: output of the ngram along with their corresponding frequencies
            Return:
                ngrams_fdist: dictionary of the ngrams with their frequencies
        '''
        with open(input_file) as f:
            input_file_ls = f.read().splitlines() 
            ngrams_ls = []
            for word in input_file_ls:
                if "." in word:
                    word = word.split(".")
                    for w_i in range(len(word) - 1):
                        w = word[w_i]
                        ngrams_ls += list(ngrams(w, self.ngram))
                else:
                    ngrams_ls += list(ngrams(word, self.ngram))

            # distribution
            self.ngrams_fdist = nltk.FreqDist(ngrams_ls)
            self.ngrams_fdist = {k: v/len(self.ngrams_fdist)/len(input_file_ls)*(10**self.P4_mfactor) for k, v in self.ngrams_fdist.items()}

            # write to output file
            if not output_csv == None:
                with open(output_csv, 'w', newline='') as outcsvfile:
                    csvwriter = csv.writer(outcsvfile, delimiter=',')
                    for ngram_chars, ngram_val in self.ngrams_fdist.items():
                        ngram_index = "0x"
                        for indx  in range(len(ngram_chars)):
                            ngram_index += ngram_chars[indx].encode('utf-8').hex()
                        csvwriter.writerow([''.join(ngram_chars), ngram_index , ngram_val])   
                    outcsvfile.close()
            return self.ngrams_fdist
    
    def sorted_ngram_fdist(self, ngrams_fdist=None) -> dict:
        '''
            Arguments:
                ngrams_fdist: dictionary of the ngrams with their frequencies
            Return:
                sorted ngrams_fdist dictionary based on the frequency values
        '''
        if ngrams_fdist == None:
            return {k: v for k, v in sorted(self.ngrams_fdist.items(), key=lambda item: item[1], reverse=False)}
        else:
            return {k: v for k, v in sorted(ngrams_fdist.items(), key=lambda item: item[1], reverse=False)}
    
    def load_domain_ngram_freq(self, input_file: str, ngram_scores: dict) -> dict:
        '''
            Arguments:
                input_file: text file of domain names
                ngram_scores: a dictionary whose keys are the ngrams, and their values is the ngram frequency (according to benign domains)
            Return:
                a dictionary whose keys are the domains and their value is the corresponding ngram
        '''
        domain_ngram_freq = {}
        with open(input_file) as f:
            i = 0
            for l in f:
                i += 1
                if i % 100000 == 0:
                    print(i, end=' ')
                if ('dga.txt' in input_file) and ("#" in l or "\n" == l): #netlab360 ommit first few lines
                    continue
                domain = l.strip()
                domain_split = domain.split(".")
                ngrams_ls = []
                for subdomain_i in range(len(domain_split)-1):
                        subdomain = domain_split[subdomain_i]
                        ngrams_ls += list(ngrams(subdomain, self.ngram))
                ngrams_val = 0
                for ng in ngrams_ls:
                    if ng in ngram_scores:
                        ngrams_val += ngram_scores[ng]
                domain_ngram_freq[domain] = ngrams_val
        
        return domain_ngram_freq
    
    def load_domain_ngram_freq_ls(self, domains: list, ngram_scores: dict) -> dict:
        '''
            Arguments:
                input_file: list of domain names
                ngram_scores: a dictionary whose keys are the ngrams, and their values is the ngram frequency (according to benign domains)
            Return:
                a dictionary whose keys are the domains and their value is the corresponding ngram
        '''
        
        domain_ngram_freq = {}
        for domain in domains:
            domain_split = domain.split(".")
            ngrams_ls = []
            for subdomain_i in range(len(domain_split)-1):
                    subdomain = domain_split[subdomain_i]
                    ngrams_ls += list(ngrams(subdomain, self.ngram))
            ngrams_val = 0
            for ng in ngrams_ls:
                if ng in ngram_scores:
                    ngrams_val += ngram_scores[ng]
            domain_ngram_freq[domain] = ngrams_val
        
        return domain_ngram_freq
    
    def get_ngram(self, domain: str, ngram_scores: dict) -> float:
        '''
            Arguments:
                domain: domain name string
                ngram_scores: dictionary of ngrams and their frequency values
            Return:
                ngram value of the domain name based on the ngram_scores
        '''
        ngram_ls = list(ngrams(domain, self.ngram))
        ngram_val = 0
        for ng in ngram_ls:
            if ng in ngram_scores:
                ngram_val += ngram_scores[ng]
        return ngram_val
    

In [3]:
print(list(ngrams("google", 2)))
print(list(ngrams("google", 3)))


[('g', 'o'), ('o', 'o'), ('o', 'g'), ('g', 'l'), ('l', 'e')]
[('g', 'o', 'o'), ('o', 'o', 'g'), ('o', 'g', 'l'), ('g', 'l', 'e')]


# ngram Class test

In [8]:
'''
    This class uses the 'Construc ngram' class above to generate bigrams and trigrams and their frequencies based 
    on the English dictionary and top 1m domains (Cisco)
'''

dataset_dir = Path("../5.1. Dataset/")

################################################### bigram
bigram_p4 = ngram_freq_P4(ngram=2, P4_mfactor=8)

# English dictionary & Top 1m domains
english_words_alpha = dataset_dir / "english_words_alpha.txt"
top_1m = dataset_dir / "top-1m.txt"

bigram_scores_alpha_output = dataset_dir / "bigram_scores_alphaEngDic.csv"
bigram_scores_top1m_output = dataset_dir / "bigram_scores_top1m.csv"
trigram_scores_alpha_output = dataset_dir / "trigram_scores_alphaEngDic.csv"
trigram_scores_top1m_output = dataset_dir / "trigram_scores_top1m.csv"
combined_bigram_scores_output = dataset_dir / "combined_bigram.csv"
combined_trigram_scores_output = dataset_dir / "combined_trigram.csv"

combined_bigram_scores = {}

bigram_scores1 = bigram_p4.create_ngram_scores(input_file=english_words_alpha, 
                                                  output_csv=bigram_scores_alpha_output) 

bigram_scores2 = bigram_p4.create_ngram_scores(input_file=top_1m,
                                                  output_csv=bigram_scores_top1m_output)


# combine top1m with english dictionary ngram frequency tuples where we take the highest value (makes sense since both of them could be used)

for e1, v1 in bigram_scores1.items():
    if e1 in bigram_scores2:
        combined_bigram_scores[e1] = max(v1, bigram_scores2[e1])
    else:
        combined_bigram_scores[e1] = v1
        
for e1, v1 in bigram_scores2.items():
    if e1 in bigram_scores1:
        combined_bigram_scores[e1] = max(v1, bigram_scores1[e1])
    else:
        combined_bigram_scores[e1] = v1        

# '''
# ################################################### trigram
trigram_p4 = ngram_freq_P4(ngram=3, P4_mfactor=10)

trigram_scores1 = trigram_p4.create_ngram_scores(input_file=english_words_alpha,
                                                    output_csv=trigram_scores_alpha_output)

trigram_scores2 = trigram_p4.create_ngram_scores(input_file=top_1m,
                                                    output_csv=trigram_scores_top1m_output)


# combine top1m with english dictionary ngram frequency tuples where we take the highest value (makes sense since both of them could be used)
combined_trigram_scores = {}
for e1, v1 in trigram_scores1.items():
    if e1 in trigram_scores2:
        combined_trigram_scores[e1] = max(v1, trigram_scores2[e1])
    else:
        combined_trigram_scores[e1] = v1
        
for e1, v1 in trigram_scores2.items():
    if e1 in trigram_scores1:
        combined_trigram_scores[e1] = max(v1, trigram_scores1[e1])
    else:
        combined_trigram_scores[e1] = v1        



# GLOBAL VARIABLES

In [46]:
events_dataset_X = []
events_dataset_y = []
dgas_X = []
dgas_y = []
ctu_X = []
ctu_y = []
events = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30,
         31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 
          60]
selected_ft_names = []

X = []
y = []
c = 1
family_to_label = {}
label_to_family = {}
indx = 1
classifiers = {"rf":[0]*len(events), "svm":[0]*len(events), "mlp":[0]*len(events), "gnb":[0]*len(events), 
                   "lr":[0]*len(events)}
context_aware_fts = []
context_less_fts = []
corr_fts = []
def reset_global_vars():
    global events_dataset_X, events_dataset_y, dgas_X, dgas_y
    global ctu_X, ctu_y, events, X, y, c, family_to_label, label_to_family, indx, classifiers
    events_dataset_X = []
    events_dataset_y = []
    dgas_X = []
    dgas_y = []
    ctu_X = []
    ctu_y = []
    X = []
    y = []
    c = 1
    family_to_label = {}
    label_to_family = {}
    indx = 1
    classifiers = {"rf":[0]*len(events), "svm":[0]*len(events), "mlp":[0]*len(events), 
                   "gnb":[0]*len(events),  "lr":[0]*len(events)}

# Features

In [47]:

def load_features(classification: int) -> (list, list):
    '''
    Function for defining the features of the classifier
    Arguments:
        classification: 0: detection of DGAs only, 1: classification of DGAs (requires more features for better accuracy)
    Return:
        context_aware_fts: context-aware features list (network features)
        context_less_fts: context-less features list (domain name features)
    '''
    global context_aware_fts
    global context_less_fts
    global corr_fts
    

    if classification == 0:
        context_aware_fts = ['uniq_pkt_cntctd_all',  'dns_reqs_all', 'nxd_iarrival']
        context_less_fts = ['bigram_score']
        corr_fts = []
    else:
        context_aware_fts = ['uniq_pkt_cntctd_all',  'dns_reqs_all', 'nxd_iarrival']
        context_less_fts = ['bigram_score', 'dml', 'nb_sub_d', 'is_valid_tld', 'has_single_sub_d', 'tld']
        corr_fts = ['nb_uniq_nxds']

    return context_aware_fts, context_less_fts, corr_fts

# Dataset Functions

In [48]:


def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn


X = []
y = []
c = 1
family_to_label = {}
label_to_family = {}
indx = 1

                
# load valid TLDs
valid_tlds = []
tld_fname = dataset_dir / tlds-alpha-by-domain.txt'
with open(tld_fname) as f_r:
    i = 0
    for l in f_r:
        if i == 0:
            i = 1
            continue
        valid_tlds.append(l.lower().strip())
        
tld_to_int = {}
i = 1
for tld in valid_tlds:
    if tld not in tld_to_int:
        tld_to_int[tld] = i
        i += 1

def qname_features(qname):
    dml = len(qname) - qname.count(".") # domain name length
    sub_domains = qname.split(".") 
    nb_sub_d = len(sub_domains) - 1 # number of subdomains
    tld = sub_domains[len(sub_domains)-1] # is valid tld

    is_valid_tld = 0 # is valid tld
    tld_indx = 0 # tld index (if valid)
    
    if tld in valid_tlds:
        is_valid_tld = 1
        tld_indx = tld_to_int[tld]
    else:
        is_valid_tld = 0

    has_single_sub_d = 0 # has a single subdomain
    for indx in range(len(sub_domains)-1):
        if len(sub_domains[indx]) == 1:
            has_single_sub_d = 1
            break
            
    nb_underscores = qname.count("_") # number of underscores


    bigram_score = bigram_p4.get_ngram(qname, combined_bigram_scores)                   
    trigram_score = trigram_p4.get_ngram(qname, combined_trigram_scores)
    

    returned_fts = {}
    returned_fts['bigram_score'] = bigram_score
    returned_fts['trigram_score'] = trigram_score
    returned_fts['dml'] = dml
    returned_fts['nb_sub_d'] = nb_sub_d
    returned_fts['is_valid_tld'] = is_valid_tld
    returned_fts['has_single_sub_d'] = has_single_sub_d
    returned_fts['tld'] = tld_indx

    return returned_fts


def features_dict_to_ls (features: str) -> list:
    '''
        Arguments:
            features: string of features  {'unique_ip_reqs':0, 'dns_reqs':0, ...}
        Returns:
            a list of the feauters' values (context-aware and context-less)
    '''
    global context_aware_fts
    global context_less_fts
    
    features = features.replace("Decimal('", "")
    features = features.replace("')", "")
    
    try:
        features = json.loads(features)
        features = ast.literal_eval(features)
    except:
        return [-1]
    
    if features['nxd_qname'] == "":
        return [-1]
    
    qname = features['nxd_qname']
    qname_ft = qname_features(qname)
    
    ls = []
    
    # context-aware features
    dns_reqs = 1
    ip_reqs = 1
    for ft in context_aware_fts:
        try:    
            ls.append(features[ft])
        except:
            continue
        
    # context-less features
    for ft in context_less_fts:
        ls.append(qname_ft[ft])

    return [ls, features]
    

def load_events_dataset(csvfile_name: str, traffic_type: str) -> (list, list):
    global events_dataset_X
    global events_dataset_y
    global events 
    global family_to_label
    global label_to_family
    global indx 
    global classification
    
    with open(csvfile_name, newline='') as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        i = 0
        for row in reader:
            i += 1

            if row[2] == '8.8.8.8':
                continue

            if traffic_type == "CTU": # benign
                if len(row) <= 3:
                    continue
                family = "benign"
                family_to_label[family] = 0
                label_to_family[0] = family

                file_name = row[0]    
            else:
                # detection only
                if classification == 0:
                    family = "dga"
                else:
                    family = row[0]


                file_name = row[1]

            if family not in family_to_label:
                family_to_label[family] = indx
                label_to_family[indx] = family
                indx += 1
            label = family_to_label[family]

            unique_nxds = set()
            for event in events:
                try:
                    if traffic_type == "CTU": # benign
                        features = "\"" + row[1 + event] + "\""
                    else:
                        features = "\"" + row[2 + event] + "\""
                except:
                    continue

                features_return = features_dict_to_ls(features)
                
                if features_return[0] == -1:
                    continue
                    
                ls = features_return[0]
                features_dict = features_return[1]
                    
                unique_nxds.add(features_dict['nxd_qname'])
                if 'nb_uniq_nxds' in corr_fts:
                    ls.append(len(unique_nxds))
                    
                #ls.append(event)

                event_index = event - 1
                if event_index >= len(events_dataset_X):
                    events_dataset_X.append([])
                    events_dataset_y.append([])
                events_dataset_X[event_index].append(ls)
                events_dataset_y[event_index].append(int(label))
                
    return events_dataset_X, events_dataset_y

def load_dataset():
    global events_dataset_X
    global events_dataset_y
    global dgas_X
    global dgas_y
    global ctu_X
    global ctu_y
    global events 
    global family_to_label
    global label_to_family
    global indx 
    global classification

    # load DGAs
    dga_fname = dataset_dir / "DGA" / 'trans_icc_newft.csv'
    load_events_dataset(dga_fname, traffic_type="DGA")

    dgas_X = copy.deepcopy(events_dataset_X)
    dgas_y = copy.deepcopy(events_dataset_y)

    x = len(dgas_X[0])
    print("DGA Dataset Length:", x)

    events_dataset_X = []
    events_dataset_y = []

    # load Benign
    if classification == 0:
        ctu_X = []
        ctu_y = []
        for i in range(42, 55): # 42, 55 !!! 
            ctu_fname = "data_no_infection_dns-" + str(i) + "_4.csv"
            ctu_rel_path = dataset_dir / "normal_ctu" / ctu_fname
            load_events_dataset(ctu_rel_path, traffic_type="CTU")

        ctu_X = copy.deepcopy(events_dataset_X)
        ctu_y = copy.deepcopy(events_dataset_y)
        print("Benign Dataset Length:", len(ctu_X[0]))

# Aggregate of all events 

In [53]:
import random 
import time

# classifiers["rf"] = [0]*len(events)
def ML(nb_trees, rf_bool=None, svm_bool=None, mlp_bool=None, gnb_bool=None, lr_bool=None):
    global classifiers 
    global classification
    global dgas_X
    global dgas_y
    
    clf = None
    for i in range(1):
        X = []
        y = []
        for event in events:
            print() 
            print("********** EVENT = ", event, "**********")
            
            if classification == 0:
                X += dgas_X[event - 1] + ctu_X[event - 1]
                y += dgas_y[event - 1] + ctu_y[event - 1]
                print("Len DGAs %f\t Len CTU %f" %(y.count(1), y.count(0)))
                               
            else:
                X += dgas_X[event - 1]
                y += dgas_y[event - 1]
                
            # RF
            # '''
            # Fitting Random Forest Classification to the training set
            
            kfold = StratifiedKFold(n_splits=5, shuffle=True)
            X_np = np.array(X)
            y_np = np.array(y)
            i = 0
            i2 = 0
            fitting_times = {'RF':[], 'SVM':[], 'GNB':[], 'MLP':[], 'LR': []}
            avg_prediction = []
            for train_ix, test_ix in kfold.split(X_np, y_np):
                X_train, X_test = X_np[train_ix], X_np[test_ix]
                y_train, y_test = y_np[train_ix], y_np[test_ix]
                
                # RF
                if rf_bool == True:
                    clf = RandomForestClassifier(n_estimators = nb_trees, criterion = 'entropy', class_weight = 'balanced')
                    # print("Fitting RF")
                    start_t = time.time()
                    clf.fit(X_train, y_train)
                    end_t = time.time()
                    fitting_times['RF'].append(end_t-start_t)

                    y_pred = clf.predict(X_test)
                    y_test = np.asarray(y_test)
                    X_test_np = np.array(X_test)
                    
                    for xs in X_test:
                        st = time.time()
                        clf.predict([xs])
                        et = time.time()
                        avg_prediction.append(et-st)
                    

                    if classifiers["rf"][event-1] == 0:
                        classifiers["rf"][event-1] = []
                    classifiers["rf"][event-1].append([accuracy_score(y_test, y_pred), 
                                                       recall_score(y_test, y_pred, average='weighted'), 
                                                        f1_score(y_test, y_pred, average='weighted'), 
                                                        precision_score(y_test, y_pred, average='weighted')])
              
                
            if rf_bool == True:
                print("Average fitting time for RF classifier at event = ", event, sum(fitting_times['RF'])/len(fitting_times['RF']))
                print("Average prediction time per sample for RF is = ", sum(avg_prediction)/len(avg_prediction))
                
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)
            print("X training/testing lengths = ", len(X_train), len(X_test))
            scaler = StandardScaler()
            X_train = scaler.fit_transform(X_train)
            X_test = scaler.transform(X_test)    
            # '''
            # SVM
            if svm_bool == True:
                clf = svm.SVC(decision_function_shape='ovo', probability=False,  C=100, kernel='linear', gamma=1)
                print("Fitting SVM")
                start_t = time.time()
                clf.fit(X_train, y_train)
                end_t = time.time()
                fitting_times['SVM'].append(end_t-start_t)
                y_pred = clf.predict(X_test)
                misclassified = np.where(y_test != clf.predict(X_test))


                # param_grid = {'C': [150], 'gamma': [1], 'kernel': ['poly']}
                # clf = GridSearchCV(SVC(class_weight = 'balanced', cache_size=700), param_grid, cv=5, refit = True, verbose = 0)
                # clf.fit(X_train, y_train)
                # y_pred = clf.predict(X_test)
                # y_test = np.asarray(y_test)
                if classifiers["svm"][event-1] == 0:
                    classifiers["svm"][event-1] = []
                classifiers["svm"][event-1].append([accuracy_score(y_test, y_pred), 
                                                   recall_score(y_test, y_pred, average='weighted'), 
                                                    f1_score(y_test, y_pred, average='weighted'), 
                                                    precision_score(y_test, y_pred, average='weighted')])
                print("Average fitting time for SVM classifier at event = ", event, sum(fitting_times['SVM'])/len(fitting_times['SVM']))
                print("Done SVM")
                    

        
            # MLP
            if mlp_bool == True:
                parameter_space = {
                    'hidden_layer_sizes': [(300, 200, 100)],
                    'activation': ['tanh', 'relu'],
                    'solver': ['sgd', 'adam'],
                    'alpha': [0.05, 0.1],
                    'learning_rate': ['constant','adaptive'],
                }
                parameter_space = {
                    'hidden_layer_sizes': [(300, 200, 100)], 'activation': ['relu'], 'solver': ['adam'], 'alpha': [0.05],
                    'learning_rate': ['adaptive'],
                }
                mlp_gs = MLPClassifier(max_iter=100)
                clf = GridSearchCV(mlp_gs, parameter_space, cv=5)

                # print("Fitting MLP")
                start_t = time.time()
                clf.fit(X_train, y_train)
                end_t = time.time()
                fitting_times['MLP'].append(end_t-start_t)
                y_pred = clf.predict(X_test)
                y_test = np.asarray(y_test)

                print("Fitting time for MLP classifier at event = ", event, sum(fitting_times['MLP'])/len(fitting_times['MLP']))

                if classifiers["mlp"][event-1] == 0:
                    classifiers["mlp"][event-1] = []
                classifiers["mlp"][event-1].append([accuracy_score(y_test, y_pred), 
                                                   recall_score(y_test, y_pred, average='weighted'), 
                                                    f1_score(y_test, y_pred, average='weighted'), 
                                                    precision_score(y_test, y_pred, average='weighted')])
                # print("Done MLP")



            # GNB
            if gnb_bool == True:
                gnb_clf = GaussianNB()
                params_NB = {'var_smoothing': np.logspace(0,-9, num=100)}
                clf = GridSearchCV(estimator=gnb_clf, 
                                 param_grid=params_NB, 
                                 cv=5,
                                 scoring='accuracy')

                # print("Fitting GNB")
                start_t = time.time()
                clf.fit(X_train, y_train)
                end_t = time.time()
                fitting_times['GNB'].append(end_t-start_t)
                y_pred = clf.predict(X_test)
                y_test = np.asarray(y_test)

                print("Fitting time for GNB classifier at event = ", event, sum(fitting_times['GNB'])/len(fitting_times['GNB']))


                if classifiers["gnb"][event-1] == 0:
                    classifiers["gnb"][event-1] = []
                classifiers["gnb"][event-1].append([accuracy_score(y_test, y_pred), 
                                                   recall_score(y_test, y_pred, average='weighted'), 
                                                    f1_score(y_test, y_pred, average='weighted'), 
                                                    precision_score(y_test, y_pred, average='weighted')])
                # print("Done GNB")



             # LR 
            if lr_bool == True:
                logModel = LogisticRegression()
                param_grid = [    
                    {'penalty' : ['l2'],
                    'C' : np.logspace(-4, 4, 3),
                    'solver' : ['lbfgs'],
                    'max_iter' : [200]
                    }
                ]
                clf = GridSearchCV(logModel, param_grid = param_grid, cv = 5, verbose=True, n_jobs=-1)
                warnings.filterwarnings("ignore")
                # print("Fitting LR")
                start_t = time.time()
                clf.fit(X_train, y_train)
                end_t = time.time()
                fitting_times['LR'].append(end_t-start_t)
                y_pred = clf.predict(X_test)
                y_test = np.asarray(y_test)

                print("Fitting time for LR classifier at event = ", event, sum(fitting_times['LR'])/len(fitting_times['LR']))


                if classifiers["lr"][event-1] == 0:
                    classifiers["lr"][event-1] = []
                classifiers["lr"][event-1].append([accuracy_score(y_test, y_pred), 
                                                   recall_score(y_test, y_pred, average='weighted'), 
                                                    f1_score(y_test, y_pred, average='weighted'), 
                                                    precision_score(y_test, y_pred, average='weighted')])
                # print("Done LR")
        
    return clf, classifiers




# Number of Families in all DGAs

In [51]:
families = {}
with open("Dataset\\DGA\\trans_icc_newft.csv", newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        if row[0] not in families:
            families[row[0]] = 0
        families[row[0]] += 1

print(len(families))
print(families)

57
{'cutwail': 5, 'ursnif': 3, 'xblocker': 2, 'qadars': 11, 'diamondfox': 6, 'zloader': 19, 'locky': 245, 'suppobox': 14, 'pushdo': 14, 'necurs': 9, 'nymaim': 10, 'pykspa': 246, 'simda': 312, 'tinba': 83, 'dircrypt': 9, 'qsnatch': 3, 'gozi': 11, 'murofet': 16, 'phorpiex': 10, 'gameover': 7, 'tofsee': 7, 'murofetweekly': 2, 'mydoom': 82, 'pykspa2': 10, 'pykspa2s': 3, 'cryptolocker': 13, 'banjori': 1, 'bazarloader': 6, 'chinad': 2, 'corebot': 6, 'dmsniff': 1, 'enviserv': 1, 'feodo': 7, 'fobber': 1, 'madmax': 1, 'makloader': 2, 'matsnu': 3, 'nymaim2': 3, 'padcrypt': 2, 'qhost': 1, 'ramdo': 2, 'ramnit': 6, 'ranbyus': 10, 'sphinx': 5, 'tinynuke': 3, 'vidrotid': 1, 'bamital': 1, 'bedep': 2, 'beebone': 3, 'downloader': 8, 'dyre': 1, 'emotet': 1, 'hesperbot': 2, 'pandabanker': 9, 'shifu': 1, 'tsifiri': 1, 'wd': 2}


# Detection, DP, RF, ['uniq_pkt_cntctd_all',  'dns_reqs_all', 'bigram_score']

In [23]:
# reset_global_vars()
# classification = 0
# context_aware_fts = ['uniq_pkt_cntctd_all',  'dns_reqs_all', 'nxd_iarrival']
# context_less_fts = ['bigram_score']
# load_dataset()
clf, classifiers = ML(nb_trees=2, rf_bool=True)
print(clf)

print("*****************************************************************************************************************")
print("*****************************************************************************************************************")
print("*****************************************************************************************************************")
print("\t\tAccuracy\tRecall\t\tF1 score\tPrecesion")
for classifier_name, vals in classifiers.items():
    try:
        print(classifier_name)
        for event_vals_i in range(len(vals)):
            acc_avg = [i[0] for i in vals[event_vals_i]]
            recall_avg = [i[1] for i in vals[event_vals_i]]
            f1_avg = [i[2] for i in vals[event_vals_i]]
            prec_avg = [i[3] for i in vals[event_vals_i]]
            print("\t", event_vals_i  + 1, "\t",
                  round(sum(acc_avg)/len(acc_avg), 6), "\t", 
                  round(sum(recall_avg)/len(recall_avg), 6), "\t", 
                  round(sum(f1_avg)/len(f1_avg), 6), "\t", 
                  round(sum(prec_avg)/len(prec_avg), 6)
            )
    except:
        continue


********** EVENT =  1 **********
Len DGAs 1247.000000	 Len CTU 990.000000

********** EVENT =  2 **********
Len DGAs 2428.000000	 Len CTU 1846.000000

********** EVENT =  3 **********
Len DGAs 3532.000000	 Len CTU 2590.000000

********** EVENT =  4 **********
Len DGAs 4606.000000	 Len CTU 3293.000000

********** EVENT =  5 **********
Len DGAs 5664.000000	 Len CTU 3917.000000

********** EVENT =  6 **********
Len DGAs 6713.000000	 Len CTU 4510.000000

********** EVENT =  7 **********
Len DGAs 7747.000000	 Len CTU 5043.000000

********** EVENT =  8 **********
Len DGAs 8777.000000	 Len CTU 5559.000000

********** EVENT =  9 **********
Len DGAs 9792.000000	 Len CTU 6030.000000

********** EVENT =  10 **********
Len DGAs 10793.000000	 Len CTU 6481.000000

********** EVENT =  11 **********
Len DGAs 11776.000000	 Len CTU 6911.000000

********** EVENT =  12 **********
Len DGAs 12722.000000	 Len CTU 7318.000000

********** EVENT =  13 **********
Len DGAs 13638.000000	 Len CTU 7696.000000

****

# Detection, CP, RF, all features

In [19]:
reset_global_vars()
classification = 0
context_aware_fts = ['uniq_pkt_cntctd_all',  'dns_reqs_all', 'nxd_iarrival']
context_less_fts = ['bigram_score', 'dml', 'nb_sub_d', 'is_valid_tld', 'tld']
corr_fts = ['nb_uniq_nxds']
load_dataset()
clf, classifiers = ML(nb_trees=30, rf_bool=True, svm_bool=False, mlp_bool=False, gnb_bool=False, lr_bool=False)


print("*****************************************************************************************************************")
print("*****************************************************************************************************************")
print("*****************************************************************************************************************")
print("\t\tAccuracy\tF1 score\tPrecesion")
for classifier_name, vals in classifiers.items():
    try:
        print(classifier_name)
        for event_vals_i in range(len(vals)):
            acc_avg = [i[0] for i in vals[event_vals_i]]
            recall_avg = [i[1] for i in vals[event_vals_i]]
            f1_avg = [i[2] for i in vals[event_vals_i]]
            prec_avg = [i[3] for i in vals[event_vals_i]]
            print("\t", event_vals_i  + 1, "\t",
                  round(sum(acc_avg)/len(acc_avg), 6), "\t", 
                  round(sum(f1_avg)/len(f1_avg), 6), "\t", 
                  round(sum(prec_avg)/len(prec_avg), 6)
            )
    except:
        continue

DGA Dataset Length: 1247
Benign Dataset Length: 990

********** EVENT =  1 **********
Len DGAs 1247.000000	 Len CTU 990.000000
Average fitting time for RF classifier at event =  1 0.0381584644317627
Average prediction time per sample for RF is =  0.0021251962821186255
X training/testing lengths =  1565 672

********** EVENT =  2 **********
Len DGAs 2428.000000	 Len CTU 1846.000000
Average fitting time for RF classifier at event =  2 0.05798993110656738
Average prediction time per sample for RF is =  0.0021029946733759452
X training/testing lengths =  2991 1283

********** EVENT =  3 **********
Len DGAs 3532.000000	 Len CTU 2590.000000
Average fitting time for RF classifier at event =  3 0.07342991828918458
Average prediction time per sample for RF is =  0.002108338610559692
X training/testing lengths =  4285 1837

********** EVENT =  4 **********
Len DGAs 4606.000000	 Len CTU 3293.000000
Average fitting time for RF classifier at event =  4 0.10036025047302247
Average prediction time pe

Average fitting time for RF classifier at event =  32 0.4679298400878906
Average prediction time per sample for RF is =  0.0021507591775339506
X training/testing lengths =  29405 12603

********** EVENT =  33 **********
Len DGAs 30092.000000	 Len CTU 12854.000000
Average fitting time for RF classifier at event =  33 0.4924917697906494
Average prediction time per sample for RF is =  0.002149428807433903
X training/testing lengths =  30062 12884

********** EVENT =  34 **********
Len DGAs 30835.000000	 Len CTU 13046.000000
Average fitting time for RF classifier at event =  34 0.49314074516296386
Average prediction time per sample for RF is =  0.0021314866067782857
X training/testing lengths =  30716 13165

********** EVENT =  35 **********
Len DGAs 31578.000000	 Len CTU 13235.000000
Average fitting time for RF classifier at event =  35 0.5032368183135987
Average prediction time per sample for RF is =  0.0021307770820127186
X training/testing lengths =  31369 13444

********** EVENT =  36

# Detection, CP, SVM, all features

In [10]:
reset_global_vars()
classification = 0
context_aware_fts = ['uniq_pkt_cntctd_all',  'dns_reqs_all', 'nxd_iarrival']
context_less_fts = ['bigram_score', 'dml', 'nb_sub_d', 'is_valid_tld', 'tld']
corr_fts = ['nb_uniq_nxds']
load_dataset()
clf, classifiers = ML(nb_trees=50, rf_bool=False, svm_bool=True, mlp_bool=False, gnb_bool=False, lr_bool=False)


print("*****************************************************************************************************************")
print("*****************************************************************************************************************")
print("*****************************************************************************************************************")
print("\t\tAccuracy\tF1 score\tPrecesion")
for classifier_name, vals in classifiers.items():
    try:
        print(classifier_name)
        for event_vals_i in range(len(vals)):
            acc_avg = [i[0] for i in vals[event_vals_i]]
            recall_avg = [i[1] for i in vals[event_vals_i]]
            f1_avg = [i[2] for i in vals[event_vals_i]]
            prec_avg = [i[3] for i in vals[event_vals_i]]
            print("\t", event_vals_i  + 1, "\t",
                  round(sum(acc_avg)/len(acc_avg), 6), "\t", 
                  round(sum(f1_avg)/len(f1_avg), 6), "\t", 
                  round(sum(prec_avg)/len(prec_avg), 6)
            )
    except:
        continue

DGA Dataset Length: 1247
Benign Dataset Length: 990

********** EVENT =  1 **********
Len DGAs 1247.000000	 Len CTU 990.000000
X training/testing lengths =  1565 672
Fitting SVM
Average fitting time for SVM classifier at event =  1 0.04351997375488281
Done SVM

********** EVENT =  2 **********
Len DGAs 2428.000000	 Len CTU 1846.000000
X training/testing lengths =  2991 1283
Fitting SVM
Average fitting time for SVM classifier at event =  2 0.17463350296020508
Done SVM

********** EVENT =  3 **********
Len DGAs 3532.000000	 Len CTU 2590.000000
X training/testing lengths =  4285 1837
Fitting SVM
Average fitting time for SVM classifier at event =  3 0.34489870071411133
Done SVM

********** EVENT =  4 **********
Len DGAs 4606.000000	 Len CTU 3293.000000
X training/testing lengths =  5529 2370
Fitting SVM
Average fitting time for SVM classifier at event =  4 0.4610919952392578
Done SVM

********** EVENT =  5 **********
Len DGAs 5664.000000	 Len CTU 3917.000000
X training/testing lengths =  6

Average fitting time for SVM classifier at event =  39 10.226795196533203
Done SVM

********** EVENT =  40 **********
Len DGAs 35276.000000	 Len CTU 14126.000000
X training/testing lengths =  34581 14821
Fitting SVM
Average fitting time for SVM classifier at event =  40 11.646390676498413
Done SVM

********** EVENT =  41 **********
Len DGAs 36003.000000	 Len CTU 14290.000000
X training/testing lengths =  35205 15088
Fitting SVM
Average fitting time for SVM classifier at event =  41 13.23831582069397
Done SVM

********** EVENT =  42 **********
Len DGAs 36729.000000	 Len CTU 14454.000000
X training/testing lengths =  35828 15355
Fitting SVM
Average fitting time for SVM classifier at event =  42 12.508460998535156
Done SVM

********** EVENT =  43 **********
Len DGAs 37454.000000	 Len CTU 14616.000000
X training/testing lengths =  36449 15621
Fitting SVM
Average fitting time for SVM classifier at event =  43 11.462424039840698
Done SVM

********** EVENT =  44 **********
Len DGAs 38178.0000

# Detection, CP, MLP, all features

In [11]:
reset_global_vars()
classification = 0
context_aware_fts = ['uniq_pkt_cntctd_all',  'dns_reqs_all', 'nxd_iarrival']
context_less_fts = ['bigram_score', 'dml', 'nb_sub_d', 'is_valid_tld', 'tld']
corr_fts = ['nb_uniq_nxds']
load_dataset()
clf, classifiers = ML(nb_trees=50, rf_bool=False, svm_bool=False, mlp_bool=True, gnb_bool=False, lr_bool=False)


print("*****************************************************************************************************************")
print("*****************************************************************************************************************")
print("*****************************************************************************************************************")
print("\t\tAccuracy\tF1 score\tPrecesion")
for classifier_name, vals in classifiers.items():
    try:
        print(classifier_name)
        for event_vals_i in range(len(vals)):
            acc_avg = [i[0] for i in vals[event_vals_i]]
            recall_avg = [i[1] for i in vals[event_vals_i]]
            f1_avg = [i[2] for i in vals[event_vals_i]]
            prec_avg = [i[3] for i in vals[event_vals_i]]
            print("\t", event_vals_i  + 1, "\t",
                  round(sum(acc_avg)/len(acc_avg), 6), "\t", 
                  round(sum(f1_avg)/len(f1_avg), 6), "\t", 
                  round(sum(prec_avg)/len(prec_avg), 6)
            )
    except:
        continue

DGA Dataset Length: 1247
Benign Dataset Length: 990

********** EVENT =  1 **********
Len DGAs 1247.000000	 Len CTU 990.000000
X training/testing lengths =  1565 672
Fitting time for MLP classifier at event =  1 5.400249481201172

********** EVENT =  2 **********
Len DGAs 2428.000000	 Len CTU 1846.000000
X training/testing lengths =  2991 1283
Fitting time for MLP classifier at event =  2 9.173179626464844

********** EVENT =  3 **********
Len DGAs 3532.000000	 Len CTU 2590.000000
X training/testing lengths =  4285 1837
Fitting time for MLP classifier at event =  3 14.525928497314453

********** EVENT =  4 **********
Len DGAs 4606.000000	 Len CTU 3293.000000
X training/testing lengths =  5529 2370
Fitting time for MLP classifier at event =  4 18.550574779510498

********** EVENT =  5 **********
Len DGAs 5664.000000	 Len CTU 3917.000000
X training/testing lengths =  6706 2875
Fitting time for MLP classifier at event =  5 22.594127655029297

********** EVENT =  6 **********
Len DGAs 6713

Fitting time for MLP classifier at event =  45 105.88385725021362

********** EVENT =  46 **********
Len DGAs 39626.000000	 Len CTU 15094.000000
X training/testing lengths =  38304 16416
Fitting time for MLP classifier at event =  46 107.87868118286133

********** EVENT =  47 **********
Len DGAs 40349.000000	 Len CTU 15249.000000
X training/testing lengths =  38918 16680
Fitting time for MLP classifier at event =  47 104.22881984710693

********** EVENT =  48 **********
Len DGAs 41072.000000	 Len CTU 15402.000000
X training/testing lengths =  39531 16943
Fitting time for MLP classifier at event =  48 100.34823775291443

********** EVENT =  49 **********
Len DGAs 41794.000000	 Len CTU 15554.000000
X training/testing lengths =  40143 17205
Fitting time for MLP classifier at event =  49 120.67501306533813

********** EVENT =  50 **********
Len DGAs 42514.000000	 Len CTU 15703.000000
X training/testing lengths =  40751 17466
Fitting time for MLP classifier at event =  50 105.00431275367737

# Detection, CP, GNB, LR, all features

In [14]:
reset_global_vars()
classification = 0
context_aware_fts = ['uniq_pkt_cntctd_all',  'dns_reqs_all', 'nxd_iarrival']
context_less_fts = ['bigram_score', 'dml', 'nb_sub_d', 'is_valid_tld', 'tld']
corr_fts = ['nb_uniq_nxds']
load_dataset()
clf, classifiers = ML(nb_trees=50, rf_bool=False, svm_bool=False, mlp_bool=False, gnb_bool=True, lr_bool=True)


print("*****************************************************************************************************************")
print("*****************************************************************************************************************")
print("*****************************************************************************************************************")
print("\t\tAccuracy\tF1 score\tPrecesion")
for classifier_name, vals in classifiers.items():
    try:
        print(classifier_name)
        for event_vals_i in range(len(vals)):
            acc_avg = [i[0] for i in vals[event_vals_i]]
            recall_avg = [i[1] for i in vals[event_vals_i]]
            f1_avg = [i[2] for i in vals[event_vals_i]]
            prec_avg = [i[3] for i in vals[event_vals_i]]
            print("\t", event_vals_i  + 1, "\t",
                  round(sum(acc_avg)/len(acc_avg), 6), "\t", 
                  round(sum(f1_avg)/len(f1_avg), 6), "\t", 
                  round(sum(prec_avg)/len(prec_avg), 6)
            )
    except:
        continue

DGA Dataset Length: 1247
Benign Dataset Length: 990

********** EVENT =  1 **********
Len DGAs 1247.000000	 Len CTU 990.000000
X training/testing lengths =  1565 672
Fitting time for GNB classifier at event =  1 0.6130061149597168
Fitting 5 folds for each of 3 candidates, totalling 15 fits
Fitting time for LR classifier at event =  1 0.035678863525390625

********** EVENT =  2 **********
Len DGAs 2428.000000	 Len CTU 1846.000000
X training/testing lengths =  2991 1283
Fitting time for GNB classifier at event =  2 0.8163800239562988
Fitting 5 folds for each of 3 candidates, totalling 15 fits
Fitting time for LR classifier at event =  2 0.05709958076477051

********** EVENT =  3 **********
Len DGAs 3532.000000	 Len CTU 2590.000000
X training/testing lengths =  4285 1837
Fitting time for GNB classifier at event =  3 0.9921472072601318
Fitting 5 folds for each of 3 candidates, totalling 15 fits
Fitting time for LR classifier at event =  3 0.07816767692565918

********** EVENT =  4 ********

Fitting time for GNB classifier at event =  27 4.224904775619507
Fitting 5 folds for each of 3 candidates, totalling 15 fits
Fitting time for LR classifier at event =  27 0.3876204490661621

********** EVENT =  28 **********
Len DGAs 26341.000000	 Len CTU 11838.000000
X training/testing lengths =  26725 11454
Fitting time for GNB classifier at event =  28 4.4742515087127686
Fitting 5 folds for each of 3 candidates, totalling 15 fits
Fitting time for LR classifier at event =  28 0.3479897975921631

********** EVENT =  29 **********
Len DGAs 27094.000000	 Len CTU 12050.000000
X training/testing lengths =  27400 11744
Fitting time for GNB classifier at event =  29 4.554851531982422
Fitting 5 folds for each of 3 candidates, totalling 15 fits
Fitting time for LR classifier at event =  29 0.3731536865234375

********** EVENT =  30 **********
Len DGAs 27847.000000	 Len CTU 12259.000000
X training/testing lengths =  28074 12032
Fitting time for GNB classifier at event =  30 4.608181476593018
F

Fitting time for LR classifier at event =  53 0.49281859397888184

********** EVENT =  54 **********
Len DGAs 45384.000000	 Len CTU 16276.000000
X training/testing lengths =  43162 18498
Fitting time for GNB classifier at event =  54 6.918859958648682
Fitting 5 folds for each of 3 candidates, totalling 15 fits
Fitting time for LR classifier at event =  54 0.5465385913848877

********** EVENT =  55 **********
Len DGAs 46100.000000	 Len CTU 16410.000000
X training/testing lengths =  43757 18753
Fitting time for GNB classifier at event =  55 7.059846878051758
Fitting 5 folds for each of 3 candidates, totalling 15 fits
Fitting time for LR classifier at event =  55 0.5316824913024902

********** EVENT =  56 **********
Len DGAs 46815.000000	 Len CTU 16544.000000
X training/testing lengths =  44351 19008
Fitting time for GNB classifier at event =  56 7.064738512039185
Fitting 5 folds for each of 3 candidates, totalling 15 fits
Fitting time for LR classifier at event =  56 0.5537455081939697



# Classification, RF

In [54]:
reset_global_vars()
classification = 1
context_aware_fts = ['uniq_pkt_cntctd_all',  'dns_reqs_all', 'nxd_iarrival']
context_less_fts = ['bigram_score', 'dml', 'nb_sub_d', 'is_valid_tld', 'tld']
corr_fts = ['nb_uniq_nxds']
load_dataset()
clf, classifiers = ML(nb_trees=100, rf_bool=True, svm_bool=False, mlp_bool=False, gnb_bool=False, lr_bool=False)


print("*****************************************************************************************************************")
print("*****************************************************************************************************************")
print("*****************************************************************************************************************")
print("\t\tAccuracy\tF1 score\tPrecesion")
for classifier_name, vals in classifiers.items():
    try:
        print(classifier_name)
        for event_vals_i in range(len(vals)):
            acc_avg = [i[0] for i in vals[event_vals_i]]
            recall_avg = [i[1] for i in vals[event_vals_i]]
            f1_avg = [i[2] for i in vals[event_vals_i]]
            prec_avg = [i[3] for i in vals[event_vals_i]]
            print("\t", event_vals_i  + 1, "\t",
                  round(sum(acc_avg)/len(acc_avg), 6), "\t", 
                  round(sum(f1_avg)/len(f1_avg), 6), "\t", 
                  round(sum(prec_avg)/len(prec_avg), 6)
            )
    except:
        continue

DGA Dataset Length: 1247

********** EVENT =  1 **********
Average fitting time for RF classifier at event =  1 0.18268771171569825
Average prediction time per sample for RF is =  0.0069348863725578105
X training/testing lengths =  872 375

********** EVENT =  2 **********
Average fitting time for RF classifier at event =  2 0.36235775947570803
Average prediction time per sample for RF is =  0.00699388597903181
X training/testing lengths =  1699 729

********** EVENT =  3 **********
Average fitting time for RF classifier at event =  3 0.574279499053955
Average prediction time per sample for RF is =  0.007136532633922919
X training/testing lengths =  2472 1060

********** EVENT =  4 **********
Average fitting time for RF classifier at event =  4 0.7444792270660401
Average prediction time per sample for RF is =  0.007127716186198782
X training/testing lengths =  3224 1382

********** EVENT =  5 **********
Average fitting time for RF classifier at event =  5 0.9040448188781738
Average pre

Average fitting time for RF classifier at event =  39 5.63020691871643
Average prediction time per sample for RF is =  0.006606487985000913
X training/testing lengths =  24177 10362

********** EVENT =  40 **********


KeyboardInterrupt: 

In [36]:
# save classification model
# pickle.dump(clf, open("trained_models\\rf_classification_60NXDs_100trees.sav", 'wb'))

# Load RF Classification Model

In [23]:
clf = pickle.load(open("trained_models\\rf_classification_30.sav", 'rb'))

In [71]:
from operator import itemgetter

top10_dgas = {}
for yi in dgas_y[0]:
    fam = label_to_family[yi]
    if fam not in top10_dgas:
        top10_dgas[fam] = 0
    top10_dgas[fam] += 1
top10_dgas = dict(sorted(top10_dgas.items(), key=itemgetter(1), reverse=True)[:10])

print(top10_dgas)


clf = pickle.load(open("trained_models\\rf_classification_30NXDs_100trees.sav", 'rb'))


df_uniq_dgafam = pd.DataFrame({})


for event in events:
    X = []
    y = []

    for family in top10_dgas:
        dga_families_acc[family] = []

    X = np.asarray(dgas_X[event - 1])
    y = np.asarray(dgas_y[event - 1])
    y_pred = clf.predict(X)
    
    for i in range(len(y)):
        family_i = label_to_family[y[i]]
        
        if family_i not in top10_dgas:
            continue

        dga_families_acc[family_i].append((y_pred[i], y[i]))
        
#     print(dga_families_acc)
    families_accuracies = {}
    for family, values in dga_families_acc.items():
        if len(values) > 0:
            acc = 0
            for v in values:
                if v[0] == v[1]:
                    acc += 1
            accuracy = acc/len(values)
            families_accuracies[family] = accuracy
            
    df_uniq_dgafam.index = list(families_accuracies.keys()) 
    df_uniq_dgafam['# of NXD=' + str(event)] = list(families_accuracies.values()) 
    
    print(df_uniq_dgafam)    
    


{'simda': 312, 'pykspa': 246, 'locky': 245, 'tinba': 83, 'mydoom': 82, 'zloader': 19, 'murofet': 16, 'suppobox': 14, 'pushdo': 14, 'cryptolocker': 13}
              # of NXD=1
zloader         0.894737
locky           0.987755
suppobox        1.000000
pushdo          1.000000
pykspa          0.890244
simda           1.000000
tinba           0.987952
murofet         1.000000
mydoom          1.000000
cryptolocker    0.846154
              # of NXD=1  # of NXD=2
zloader         0.894737    1.000000
locky           0.987755    0.995918
suppobox        1.000000    1.000000
pushdo          1.000000    0.928571
pykspa          0.890244    0.995935
simda           1.000000    1.000000
tinba           0.987952    1.000000
murofet         1.000000    0.933333
mydoom          1.000000    1.000000
cryptolocker    0.846154    1.000000
              # of NXD=1  # of NXD=2  # of NXD=3
zloader         0.894737    1.000000    0.928571
locky           0.987755    0.995918    0.995918
suppobox        1.00

              # of NXD=1  # of NXD=2  # of NXD=3  # of NXD=4  # of NXD=5  \
zloader         0.894737    1.000000    0.928571    0.928571    0.928571   
locky           0.987755    0.995918    0.995918    1.000000    0.991770   
suppobox        1.000000    1.000000    0.928571    1.000000    1.000000   
pushdo          1.000000    0.928571    1.000000    1.000000    0.928571   
pykspa          0.890244    0.995935    1.000000    1.000000    0.995935   
simda           1.000000    1.000000    0.996795    1.000000    1.000000   
tinba           0.987952    1.000000    0.977273    0.974359    1.000000   
murofet         1.000000    0.933333    1.000000    1.000000    1.000000   
mydoom          1.000000    1.000000    1.000000    0.925926    0.960000   
cryptolocker    0.846154    1.000000    0.846154    1.000000    1.000000   

              # of NXD=6  # of NXD=7  # of NXD=8  # of NXD=9  # of NXD=10  \
zloader         0.928571    0.928571    0.928571    0.928571     0.928571   
locky    

              # of NXD=1  # of NXD=2  # of NXD=3  # of NXD=4  # of NXD=5  \
zloader         0.894737    1.000000    0.928571    0.928571    0.928571   
locky           0.987755    0.995918    0.995918    1.000000    0.991770   
suppobox        1.000000    1.000000    0.928571    1.000000    1.000000   
pushdo          1.000000    0.928571    1.000000    1.000000    0.928571   
pykspa          0.890244    0.995935    1.000000    1.000000    0.995935   
simda           1.000000    1.000000    0.996795    1.000000    1.000000   
tinba           0.987952    1.000000    0.977273    0.974359    1.000000   
murofet         1.000000    0.933333    1.000000    1.000000    1.000000   
mydoom          1.000000    1.000000    1.000000    0.925926    0.960000   
cryptolocker    0.846154    1.000000    0.846154    1.000000    1.000000   

              # of NXD=6  # of NXD=7  # of NXD=8  # of NXD=9  # of NXD=10  \
zloader         0.928571    0.928571    0.928571    0.928571     0.928571   
locky    

              # of NXD=1  # of NXD=2  # of NXD=3  # of NXD=4  # of NXD=5  \
zloader         0.894737    1.000000    0.928571    0.928571    0.928571   
locky           0.987755    0.995918    0.995918    1.000000    0.991770   
suppobox        1.000000    1.000000    0.928571    1.000000    1.000000   
pushdo          1.000000    0.928571    1.000000    1.000000    0.928571   
pykspa          0.890244    0.995935    1.000000    1.000000    0.995935   
simda           1.000000    1.000000    0.996795    1.000000    1.000000   
tinba           0.987952    1.000000    0.977273    0.974359    1.000000   
murofet         1.000000    0.933333    1.000000    1.000000    1.000000   
mydoom          1.000000    1.000000    1.000000    0.925926    0.960000   
cryptolocker    0.846154    1.000000    0.846154    1.000000    1.000000   

              # of NXD=6  # of NXD=7  # of NXD=8  # of NXD=9  # of NXD=10  \
zloader         0.928571    0.928571    0.928571    0.928571     0.928571   
locky    

ValueError: Length mismatch: Expected axis has 10 elements, new values have 9 elements

# Classification, MLP

In [11]:
reset_global_vars()
classification = 1
context_aware_fts = ['uniq_pkt_cntctd_all',  'dns_reqs_all', 'nxd_iarrival']
context_less_fts = ['bigram_score', 'dml', 'nb_sub_d', 'is_valid_tld', 'tld']
corr_fts = ['nb_uniq_nxds']
load_dataset()
clf, classifiers = ML(nb_trees=100, rf_bool=False, svm_bool=False, mlp_bool=True, gnb_bool=False, lr_bool=False)


print("*****************************************************************************************************************")
print("*****************************************************************************************************************")
print("*****************************************************************************************************************")
print("\t\tAccuracy\tF1 score\tPrecesion")
for classifier_name, vals in classifiers.items():
    try:
        print(classifier_name)
        for event_vals_i in range(len(vals)):
            acc_avg = [i[0] for i in vals[event_vals_i]]
            recall_avg = [i[1] for i in vals[event_vals_i]]
            f1_avg = [i[2] for i in vals[event_vals_i]]
            prec_avg = [i[3] for i in vals[event_vals_i]]
            print("\t", event_vals_i  + 1, "\t",
                  round(sum(acc_avg)/len(acc_avg), 6), "\t", 
                  round(sum(f1_avg)/len(f1_avg), 6), "\t", 
                  round(sum(prec_avg)/len(prec_avg), 6)
            )
    except:
        continue

DGA Dataset Length: 1247

********** EVENT =  1 **********
X training/testing lengths =  872 375
Fitting time for MLP classifier at event =  1 3.8896563053131104

********** EVENT =  2 **********
X training/testing lengths =  1699 729
Fitting time for MLP classifier at event =  2 7.118627071380615

********** EVENT =  3 **********
X training/testing lengths =  2472 1060
Fitting time for MLP classifier at event =  3 10.444093465805054

********** EVENT =  4 **********
X training/testing lengths =  3224 1382
Fitting time for MLP classifier at event =  4 13.70009708404541

********** EVENT =  5 **********
X training/testing lengths =  3964 1700
Fitting time for MLP classifier at event =  5 16.805643796920776

********** EVENT =  6 **********
X training/testing lengths =  4699 2014
Fitting time for MLP classifier at event =  6 20.50827193260193

********** EVENT =  7 **********
X training/testing lengths =  5422 2325
Fitting time for MLP classifier at event =  7 25.03233790397644

********

Fitting time for MLP classifier at event =  59 152.31098580360413

********** EVENT =  60 **********
X training/testing lengths =  34769 14901
Fitting time for MLP classifier at event =  60 154.91836595535278
*****************************************************************************************************************
*****************************************************************************************************************
*****************************************************************************************************************
		Accuracy	F1 score	Precesion
rf
svm
mlp
	 1 	 0.813333 	 0.788036 	 0.786476
	 2 	 0.843621 	 0.81998 	 0.810883
	 3 	 0.880189 	 0.864163 	 0.861249
	 4 	 0.889291 	 0.87579 	 0.87599
	 5 	 0.902353 	 0.895165 	 0.898823
	 6 	 0.904171 	 0.897979 	 0.902885
	 7 	 0.908817 	 0.901704 	 0.902184
	 8 	 0.904328 	 0.891585 	 0.891189
	 9 	 0.912526 	 0.90415 	 0.903022
	 10 	 0.912909 	 0.906386 	 0.911055
	 11 	 0.919615 	 0.911418 	 0.909909
	 1

# Classification, MLP2

In [21]:
reset_global_vars()
classification = 1
context_aware_fts = ['uniq_pkt_cntctd_all',  'dns_reqs_all', 'nxd_iarrival']
context_less_fts = ['bigram_score', 'dml', 'nb_sub_d', 'is_valid_tld', 'tld']
corr_fts = ['nb_uniq_nxds']
load_dataset()
clf, classifiers = ML(nb_trees=100, rf_bool=False, svm_bool=False, mlp_bool=True, gnb_bool=False, lr_bool=False)


print("*****************************************************************************************************************")
print("*****************************************************************************************************************")
print("*****************************************************************************************************************")
print("\t\tAccuracy\tF1 score\tPrecesion")
for classifier_name, vals in classifiers.items():
    try:
        print(classifier_name)
        for event_vals_i in range(len(vals)):
            acc_avg = [i[0] for i in vals[event_vals_i]]
            recall_avg = [i[1] for i in vals[event_vals_i]]
            f1_avg = [i[2] for i in vals[event_vals_i]]
            prec_avg = [i[3] for i in vals[event_vals_i]]
            print("\t", event_vals_i  + 1, "\t",
                  round(sum(acc_avg)/len(acc_avg), 6), "\t", 
                  round(sum(f1_avg)/len(f1_avg), 6), "\t", 
                  round(sum(prec_avg)/len(prec_avg), 6)
            )
    except:
        continue

DGA Dataset Length: 1247

********** EVENT =  1 **********
X training/testing lengths =  872 375
Fitting time for MLP classifier at event =  1 7.777456760406494

********** EVENT =  2 **********
X training/testing lengths =  1699 729
Fitting time for MLP classifier at event =  2 13.458893537521362

********** EVENT =  3 **********
X training/testing lengths =  2472 1060
Fitting time for MLP classifier at event =  3 20.268304347991943

********** EVENT =  4 **********
X training/testing lengths =  3224 1382
Fitting time for MLP classifier at event =  4 28.141533613204956

********** EVENT =  5 **********
X training/testing lengths =  3964 1700
Fitting time for MLP classifier at event =  5 33.273439168930054

********** EVENT =  6 **********
X training/testing lengths =  4699 2014
Fitting time for MLP classifier at event =  6 39.09736204147339

********** EVENT =  7 **********
X training/testing lengths =  5422 2325
Fitting time for MLP classifier at event =  7 45.88003587722778

*******

Fitting time for MLP classifier at event =  59 290.1665983200073

********** EVENT =  60 **********
X training/testing lengths =  34769 14901
Fitting time for MLP classifier at event =  60 299.54137539863586
*****************************************************************************************************************
*****************************************************************************************************************
*****************************************************************************************************************
		Accuracy	F1 score	Precesion
rf
svm
mlp
	 1 	 0.861333 	 0.855081 	 0.860207
	 2 	 0.866941 	 0.861362 	 0.872604
	 3 	 0.877358 	 0.868312 	 0.867806
	 4 	 0.888567 	 0.875509 	 0.877642
	 5 	 0.917059 	 0.912494 	 0.91661
	 6 	 0.905164 	 0.898719 	 0.902128
	 7 	 0.904946 	 0.895289 	 0.896398
	 8 	 0.921033 	 0.915266 	 0.920732
	 9 	 0.927161 	 0.922565 	 0.925467
	 10 	 0.933601 	 0.929819 	 0.93122
	 11 	 0.93405 	 0.926989 	 0.928225
	 1

# Classification, SVM

In [19]:
reset_global_vars()
classification = 1
context_aware_fts = ['uniq_pkt_cntctd_all',  'dns_reqs_all', 'nxd_iarrival']
context_less_fts = ['bigram_score', 'dml', 'nb_sub_d', 'is_valid_tld', 'tld']
corr_fts = ['nb_uniq_nxds']
load_dataset()
clf, classifiers = ML(nb_trees=100, rf_bool=False, svm_bool=True, mlp_bool=False, gnb_bool=False, lr_bool=False)


print("*****************************************************************************************************************")
print("*****************************************************************************************************************")
print("*****************************************************************************************************************")
print("\t\tAccuracy\tF1 score\tPrecesion")
for classifier_name, vals in classifiers.items():
    try:
        print(classifier_name)
        for event_vals_i in range(len(vals)):
            acc_avg = [i[0] for i in vals[event_vals_i]]
            recall_avg = [i[1] for i in vals[event_vals_i]]
            f1_avg = [i[2] for i in vals[event_vals_i]]
            prec_avg = [i[3] for i in vals[event_vals_i]]
            print("\t", event_vals_i  + 1, "\t",
                  round(sum(acc_avg)/len(acc_avg), 6), "\t", 
                  round(sum(f1_avg)/len(f1_avg), 6), "\t", 
                  round(sum(prec_avg)/len(prec_avg), 6)
            )
    except:
        continue

DGA Dataset Length: 1247

********** EVENT =  1 **********
X training/testing lengths =  872 375
Fitting SVM
Average fitting time for SVM classifier at event =  1 0.06504511833190918
Done SVM

********** EVENT =  2 **********
X training/testing lengths =  1699 729
Fitting SVM
Average fitting time for SVM classifier at event =  2 0.15380334854125977
Done SVM

********** EVENT =  3 **********
X training/testing lengths =  2472 1060
Fitting SVM
Average fitting time for SVM classifier at event =  3 0.5635113716125488
Done SVM

********** EVENT =  4 **********
X training/testing lengths =  3224 1382
Fitting SVM
Average fitting time for SVM classifier at event =  4 0.5761573314666748
Done SVM

********** EVENT =  5 **********
X training/testing lengths =  3964 1700
Fitting SVM
Average fitting time for SVM classifier at event =  5 1.0226356983184814
Done SVM

********** EVENT =  6 **********
X training/testing lengths =  4699 2014
Fitting SVM
Average fitting time for SVM classifier at event =

Average fitting time for SVM classifier at event =  49 17.79087233543396
Done SVM

********** EVENT =  50 **********
X training/testing lengths =  29759 12755
Fitting SVM
Average fitting time for SVM classifier at event =  50 20.18281078338623
Done SVM

********** EVENT =  51 **********
X training/testing lengths =  30262 12970
Fitting SVM
Average fitting time for SVM classifier at event =  51 23.481277227401733
Done SVM

********** EVENT =  52 **********
X training/testing lengths =  30765 13185
Fitting SVM
Average fitting time for SVM classifier at event =  52 25.632415771484375
Done SVM

********** EVENT =  53 **********
X training/testing lengths =  31266 13401
Fitting SVM
Average fitting time for SVM classifier at event =  53 16.847151517868042
Done SVM

********** EVENT =  54 **********
X training/testing lengths =  31768 13616
Fitting SVM
Average fitting time for SVM classifier at event =  54 18.419748067855835
Done SVM

********** EVENT =  55 **********
X training/testing lengt

# Classification, GNB, LR

In [11]:
reset_global_vars()
classification = 1
context_aware_fts = ['uniq_pkt_cntctd_all',  'dns_reqs_all', 'nxd_iarrival']
context_less_fts = ['bigram_score', 'dml', 'nb_sub_d', 'is_valid_tld', 'tld']
corr_fts = ['nb_uniq_nxds']
load_dataset()
clf, classifiers = ML(nb_trees=100, rf_bool=False, svm_bool=False, mlp_bool=False, gnb_bool=True, lr_bool=True)


print("*****************************************************************************************************************")
print("*****************************************************************************************************************")
print("*****************************************************************************************************************")
print("\t\tAccuracy\tF1 score\tPrecesion")
for classifier_name, vals in classifiers.items():
    try:
        print(classifier_name)
        for event_vals_i in range(len(vals)):
            acc_avg = [i[0] for i in vals[event_vals_i]]
            recall_avg = [i[1] for i in vals[event_vals_i]]
            f1_avg = [i[2] for i in vals[event_vals_i]]
            prec_avg = [i[3] for i in vals[event_vals_i]]
            print("\t", event_vals_i  + 1, "\t",
                  round(sum(acc_avg)/len(acc_avg), 6), "\t", 
                  round(sum(f1_avg)/len(f1_avg), 6), "\t", 
                  round(sum(prec_avg)/len(prec_avg), 6)
            )
    except:
        continue

DGA Dataset Length: 1247

********** EVENT =  1 **********
X training/testing lengths =  872 375
Fitting time for GNB classifier at event =  1 1.9171130657196045
Fitting 5 folds for each of 3 candidates, totalling 15 fits
Fitting time for LR classifier at event =  1 2.9611268043518066

********** EVENT =  2 **********
X training/testing lengths =  1699 729
Fitting time for GNB classifier at event =  2 2.4813759326934814
Fitting 5 folds for each of 3 candidates, totalling 15 fits
Fitting time for LR classifier at event =  2 2.023576259613037

********** EVENT =  3 **********
X training/testing lengths =  2472 1060
Fitting time for GNB classifier at event =  3 2.8170268535614014
Fitting 5 folds for each of 3 candidates, totalling 15 fits
Fitting time for LR classifier at event =  3 3.7505862712860107

********** EVENT =  4 **********
X training/testing lengths =  3224 1382
Fitting time for GNB classifier at event =  4 3.130629062652588
Fitting 5 folds for each of 3 candidates, totalling 

Fitting time for GNB classifier at event =  32 10.02155613899231
Fitting 5 folds for each of 3 candidates, totalling 15 fits
Fitting time for LR classifier at event =  32 35.61806631088257

********** EVENT =  33 **********
X training/testing lengths =  21064 9028
Fitting time for GNB classifier at event =  33 10.47009015083313
Fitting 5 folds for each of 3 candidates, totalling 15 fits
Fitting time for LR classifier at event =  33 37.81044411659241

********** EVENT =  34 **********
X training/testing lengths =  21584 9251
Fitting time for GNB classifier at event =  34 10.32349157333374
Fitting 5 folds for each of 3 candidates, totalling 15 fits
Fitting time for LR classifier at event =  34 37.89457082748413

********** EVENT =  35 **********
X training/testing lengths =  22104 9474
Fitting time for GNB classifier at event =  35 10.20591425895691
Fitting 5 folds for each of 3 candidates, totalling 15 fits
Fitting time for LR classifier at event =  35 37.571157932281494

********** EVE